Used the library "tweepy".In order to access Twitter Streaming API, we need to get 4 pieces of information from Twitter: API key, API secret, Access token and Access token secret.To get these create Twitter Developer Account.Select the userID.

In [ ]:
import tweepy
access_token = "XXXXXXXXXXXXXXXXXXXXXXX"
access_token_secret = "XXXXXXXXXXXXXXXXXXXXXXX"
consumer_key = "XXXXXXXXXXXXXXXXXXXXXXX"
consumer_secret = "XXXXXXXXXXXXXXXXXXXXXXX"
userID = "@midasIIITD""

Extract the latest 200 tweets using api.user_timeline. We need to authorize our Twitter credentials

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
tweets = api.user_timeline(screen_name=userID, 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full text else only the first 140 words are extracted
                           tweet_mode = 'extended')

To extract as many as possible

In [ ]:
all_tweets = []
all_tweets.extend(tweets)
oldest_id = tweets[-1].id

while True:
    tweets = api.user_timeline(screen_name=userID,count=200,include_rts = False,max_id = oldest_id - 1,tweet_mode = 'extended')
    if len(tweets) == 0:
        break
    oldest_id = tweets[-1].id
    all_tweets.extend(tweets)
    print('N of tweets downloaded till now {}'.format(len(all_tweets)))

Transform the tweepy tweets into an array.

In [ ]:
Thetweet = [[tweet.id_str, 
              tweet.created_at, 
              tweet.favorite_count, 
              tweet.retweet_count,
              tweet.full_text.encode("utf-8").decode("utf-8")#compromise character encoding that can be as compact as ASCII.Uses 8 bit to encode a character 
              ,tweet.entities,] 
             for idx,tweet in enumerate(all_tweets) #adds a counter to an iterable ]

tweet.entities gives us a list nested with list and dictionary.We have to look for media and then media type in order to count number of photos in a tweet

In [ ]:
count_photo=[]
count=0
for i in range (0,len(Thetweet)):
    if 'media' in Thetweet[i][5]:
        if Thetweet[i][5]["media"][0]["type"] == 'photo':
            count=count+1
            count_photo.append(count)
            count=0
        else:
            count_photo.append("None")
    else:
        count_photo.append("None")

count_photo gives an array which contains information of number of photos in a tweet. last column of TheTweet is of no use.So,delete it and stack count_photo as last column

In [ ]:
import numpy as np 
TheTweet=np.delete(TheTweet, -1, axis=1)
TheTweet=np.column_stack((count_photo,size_photo))

Storing it in jason file.We can save it in csv file by"df.to_csv('TheTweet.csv')"

In [ ]:
from pandas import DataFrame
df = DataFrame(TheTweet,columns=["id","created_at","favorite_count","retweet_count", "text","Number of photo in a tweet"])
df.to_json('TheTweet.json',orient="table")